In [ ]:
import numpy as np
def sample():
    # returns a state x that is sampled uniformly randomly from the domain
    while(1):
        x = np.random.uniform(-9.4,9.4,1)
        y = np.random.uniform(-9.4,9.4,1)
        point = (x[0],y[0])
        #print(point)
        
        if (((-6.6 <= x[0])&(x[0]<= 0.6))&((-5.6<=y[0])&(y[0]<=-3.4))|((3.4 <=x[0])& (x[0]<= 5.6))&((-4.6<=y[0])&(y[0]<=10))):
            x = np.random.uniform(-9.4,9.4,1)
            y = np.random.uniform(-9.4,9.4,1)
            point = (x[0],y[0])   
        else:
            break
    return point

In [ ]:
def steer(point1, point2):
    # returns the optimal control trajectory of traveling from x1 to x2
    x1,y1 = point1[0],point1[1]
    x2,y2 = point2[0],point2[1]
    theta = np.arctan2(y2-y1,x2-x1)
    new_x = np.cos(theta)
    new_y = np.sin(theta)
    new_point = (new_x+x1,new_y+y1)
    return new_point

# start = (5,5)
# ans = steer(start,random_point)
# print(ans)

In [ ]:
def is_in_collision(point):
    # returns True if state x of the robot is incollision with any of the obstacles
    (x,y) = point 
    if (((-6.6 <= x)&(x<= 0.6))&((-5.6<=y)&(y<=-3.4))|((3.4 <=x)& (x<= 5.6))&((-4.6<=y)&(y<=10))):
        #print('This is in collision')
        return True
    else:
        return False

In [ ]:
from scipy.spatial import distance
def nearest(node,tree):
    # finds a node in the tree that is closest to the state x (closest in what metric?)
    nearest_node = None
    dmin = np.inf
    #print(tree)
    for point in tree:
        #print(point)
        xp,yp = point[0],point[1]
        d = distance.euclidean((xp,yp),node)
        if d<dmin:
            dmin = d
            nearest_node = point   
    return nearest_node

In [ ]:
def connect(node,nearest_node,tree):
    # add state x to the tree
    # cost of state x = cost of the parent + cost of steer(parent, x)
    #print('Node;',node)
    (x,y) = node
    #print(tree)
    tree_points = tree[:,0:2]
    #print(tree_points)
    #print('tree points',tree_points)
    #print(nearest_node)
    parent_node = np.where(np.all(tree == nearest_node,axis = 1))[0][0]
    new_node = np.array([x,y,int(parent_node)])
    tree = np.vstack([tree,new_node])
    return tree,new_node


In [ ]:
def cost(node,tree):
    parent_indices = []
    node1 = node
    while(1):
        (x,y,parent) = node1
        parent_indices.append(parent)
        #print("parent", parent)
        node1 = tree[int(parent)]
        if parent == 0:
            break
    #print('tree',tree)
    #print('node',node)
    present_node = np.where(np.all(tree==node,axis=1))[0][0]
    path_nodes = [p for p in reversed(parent_indices)]
    path_nodes.append(present_node)
    cost = 0
    #print('Path nodes:',path_nodes)
    for index in path_nodes:
        child = tree[int(index)]
        #print('Iteration:',index,'Child node:',child)
        parent_index = child[2]
        parent = tree[int(parent_index)]
        #print('Iteration:',index,'Parent node:',parent)
        cost+= distance.euclidean(parent[0:2],child[0:2])
    return cost

In [ ]:
def rewire(node,tree):
    # rewire all nodes in the tree within the O(gamma (log n/n)ˆ{1/d}} ball
    # near the state x, update the costs of all rewired neighbor
    
    graph_size = len(tree)
    ball_radius = 3
    #np.log(graph_size)/graph_size
    #print(ball_radius)
    new_tree = np.copy(tree)
    rewire_list = []
    for another_node in tree:
        (xa,ya,pa) = another_node
        (x,y,p) = node
        if distance.euclidean((x,y),(xa,ya)) <= ball_radius:
            #print('hi')
            #print(another_node)
            rewire_list.append(another_node)
    #print('Rewire list:',rewire_list)       
    for anode in rewire_list:
        # CHECK COST OF THE NODE
        #print('anode:',anode)
        cost_of_anode = cost(anode,tree)

        #CHANGE PARENT OF  ANODE TO GIVEN NODE 
        temp_anode = np.copy(anode)
        (xt,yt,pt) = temp_anode

        node_id = np.where(np.all(tree == node,axis = 1))
        #print(node_id)
        node_id = node_id[0][0]
        temp_anode = np.array([xt,yt,node_id])
        temp_tree = np.vstack([tree,temp_anode])

        #COMPUTE UPDATED NODE COST
        cost_of_temp_anode = cost(temp_anode,temp_tree)

        if cost_of_temp_anode < cost_of_anode:
            # WE MUST REWIRE THE ANODE TO NODE AS PARENT AFTER CHECKING FOR COLLISION IN BETWEEN THEM
            
            #-----------------------------------------------------------------------------------
            (x1,y1,p1) = node
            (x2,y2,p2) = temp_anode

            dist = distance.euclidean((x1,y1),(x2,y2))
            angle = np.arctan2(y2-y1,x2-x1)
            lis1 = np.linspace(0,dist,num=300)

            status = []

            for d in lis1:
                pt = np.array([x1+d*np.cos(angle),y1+d*np.sin(angle)])
                status.append(is_in_collision(pt))


            Fals = [False for i in range(len(status))]

            if status == Fals:  
                #-----------------------------------------------------------------------------------
                node_location = np.where(np.all(tree == anode,axis = 1))[0][0]
                tree[node_location] = temp_anode      
    
    return tree

In [ ]:
def goal_cost(node,tree):
    parent_indices = []
    node1 = node
    while(1):
        (x,y,parent) = node1
        parent_indices.append(parent)
        #print("parent", parent)
        node1 = tree[int(parent)]
        if parent == 0:
            break
    #print('tree',tree)
    #print('node',node)
    present_node = np.where(np.all(tree==node,axis=1))[0][0]
    path_nodes = [p for p in reversed(parent_indices)]
    path_nodes.append(present_node)
    cost = 0
    #print('Path nodes:',path_nodes)
    for index in path_nodes:
        child = tree[int(index)]
        #print('Iteration:',index,'Child node:',child)
        parent_index = child[2]
        parent = tree[int(parent_index)]
        #print('Iteration:',index,'Parent node:',parent)
        cost+= distance.euclidean(parent[0:2],child[0:2])
    return path_nodes,cost

In [ ]:
def goal_search(final_tree):
    
    # DEFINE A BALL OF RADIUS 0.5 AROUND THE GOAL NODE AND LIST ALL NODES IN THE REGION
    
    goal_nodes = []
    goal = (8,8)
    
    for gnode in final_tree:
        (x,y,p) = gnode
        lhs = (x-8)**2 + (y-8)**2
        if lhs<0.25:
            goal_nodes.append(gnode)  
    possible_goals = np.asarray(goal_nodes)
    
    if len(goal_nodes)==0:
        
        return np.inf,None
    
    else:
        
        # IF GOAL FOUND, BACK TRACK TO START POINT AND FIND OPTIMAL GOAL PATH AND COST
        min_cost = np.inf
        target = None
        for agoal in possible_goals:
            costa = cost(agoal,final_tree)
            if costa < min_cost:
                min_cost = costa
                target = agoal
        # RETURN BEST PATH AND OPTIMAL COST, IF THEY EXIST
        best_path,best_cost = goal_cost(target,final_tree)
        return best_cost,best_path

best_cost,best_path = goal_search(final_tree)

In [ ]:
# INITIALISE TREE

tree = np.array([[0,0,0]])
#print(tree)

In [ ]:
best_cost_list = []

for i in range(10000):
        
    node = sample()
    nearest_node = nearest(node,tree)
    new_node = steer(nearest_node,node)
    
    (x1,y1,p1) = nearest_node
    (x2,y2) = new_node

    dist = distance.euclidean((x1,y1),(x2,y2))
    angle = np.arctan2(y2-y1,x2-x1)
    lis1 = np.linspace(0,dist,num=300)
    
    status = []

    for d in lis1:
        pt = np.array([x1+d*np.cos(angle),y1+d*np.sin(angle)])
        status.append(is_in_collision(pt))
    #print(len(status))
    
    Fals = [False for i in range(len(status))]
    #print(Fals)
    
    if status == Fals:  
        
        # CONNECT NEW_NODE TO TREE
        tree,new_n = connect(new_node,nearest_node,tree)
        
        # REWIRE NEW_N TO FIND BETTER PARENTS
        tree = rewire(new_n,tree)
        #-----------------------------------------------------------------------
        
        # DONE CONNECTING NEW_N TO TREE WITH BEST PARENT POSSIBLE
        
        #NOW
        
        # CHECK IF NEW_N IS A GOOD PARENT FOR OTHERS AFTER REWIRING NEW_N
        
        #------------------------------------------------------------------------
        
        ball_radius = 3
        rewire_list = []
        
        for another_node in tree:
            (xa,ya,pa) = another_node
            (x,y,p) = new_n
            if distance.euclidean((x,y),(xa,ya)) <= ball_radius:
                rewire_list.append(another_node)
                
        for anode in rewire_list:
            
            # CHECK COST OF THE NODE
            cost_of_anode = cost(anode,tree)

            #CHANGE PARENT OF  ANODE TO GIVEN NODE 
            temp_anode = np.copy(anode)
            (xt,yt,pt) = temp_anode

            node_id = np.where(np.all(tree == new_n,axis = 1))
            #print(node_id)
            node_id = node_id[0][0]
            temp_anode = np.array([xt,yt,node_id])
            temp_tree = np.vstack([tree,temp_anode])

            #COMPUTE UPDATED NODE COST
            cost_of_temp_anode = cost(temp_anode,temp_tree)

            if cost_of_temp_anode < cost_of_anode:
                # WE MUST REWIRE THE ANODE TO NODE AS PARENT
                
                #-----------------------------------------------------------------------------------
                (x1,y1,p1) = new_n
                (x2,y2,p2) = temp_anode

                dist = distance.euclidean((x1,y1),(x2,y2))
                angle = np.arctan2(y2-y1,x2-x1)
                lis1 = np.linspace(0,dist,num=300)

                status = []

                for d in lis1:
                    pt = np.array([x1+d*np.cos(angle),y1+d*np.sin(angle)])
                    status.append(is_in_collision(pt))


                Fals = [False for i in range(len(status))]

                if status == Fals:  
                    #-----------------------------------------------------------------------------------
                    node_location = np.where(np.all(tree == anode,axis = 1))[0][0]

                    tree[node_location] = temp_anode
        
    best_cost,best_path = goal_search(tree)
    best_cost_list.append(best_cost)
    if (i%100 == 0):
        print('Iteration:',i,'Best Cost:',best_cost)

In [ ]:
final_tree = np.copy(tree)

In [ ]:
plt.plot(best_cost_list)

In [ ]:
def goal_cost(node,tree):
    parent_indices = []
    node1 = node
    while(1):
        (x,y,parent) = node1
        parent_indices.append(parent)
        #print("parent", parent)
        node1 = tree[int(parent)]
        if parent == 0:
            break
    #print('tree',tree)
    #print('node',node)
    present_node = np.where(np.all(tree==node,axis=1))[0][0]
    path_nodes = [p for p in reversed(parent_indices)]
    path_nodes.append(present_node)
    cost = 0
    #print('Path nodes:',path_nodes)
    for index in path_nodes:
        child = tree[int(index)]
        #print('Iteration:',index,'Child node:',child)
        parent_index = child[2]
        parent = tree[int(parent_index)]
        #print('Iteration:',index,'Parent node:',parent)
        cost+= distance.euclidean(parent[0:2],child[0:2])
    return path_nodes,cost

In [ ]:
def goal_search(final_tree):
    
    # DEFINE A BALL OF RADIUS 0.5 AROUND THE GOAL NODE AND LIST ALL NODES IN THE REGION
    
    goal_nodes = []
    goal = (8,8)
    
    for gnode in final_tree:
        (x,y,p) = gnode
        lhs = (x-8)**2 + (y-8)**2
        if lhs<0.25:
            goal_nodes.append(gnode)  
    possible_goals = np.asarray(goal_nodes)
    
    if len(goal_nodes)==0:
        
        return np.inf,None
    
    else:
        
        # IF GOAL FOUND, BACK TRACK TO START POINT AND FIND OPTIMAL GOAL PATH AND COST
        min_cost = np.inf
        target = None
        for agoal in possible_goals:
            costa = cost(agoal,final_tree)
            if costa < min_cost:
                min_cost = costa
                target = agoal
        # RETURN BEST PATH AND OPTIMAL COST, IF THEY EXIST
        best_path,best_cost = goal_cost(target,final_tree)
        return best_cost,best_path

# BEST COST TO GOAL

best_cost,best_path = goal_search(final_tree)
print(best_cost)

In [ ]:
path_x = []
path_y = []
for i in best_path:
    temp = final_tree[int(i)]
    path_x.append(temp[0])
    path_y.append(temp[1])

# PATH NODES
path = np.asarray([np.asarray(path_x),np.asarray(path_y)])

In [ ]:
import matplotlib.pyplot as plt

a1=[4,9]
b1=[5,9]
c1=[5,-4]
d1=[4,-4]
width = c1[0] - a1[0]
height = d1[1] - a1[1]
lims = (0, 10)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

fig1 = plt.figure()
ax1 = fig1.add_subplot(111, aspect='equal')
ax1.add_patch(patches.Rectangle(a1, width, height))

a=[-6,-4]
b=[0,-4]
c=[0,-5]
d=[-6,-5]
width = c[0] - a[0]
height = d[1] - a[1]
lims = (0, 10)

#fig1 = plt.figure()
#ax1 = fig1.add_subplot(111, aspect='equal')
ax1.add_patch(patches.Rectangle(a, width, height))

ax1.plot(path_x,path_y,"-",c= 'r')
plt.ylim((-10, 10)) 
plt.xlim((-10, 10))
plt.show()

In [ ]:

a1=[4,9]
b1=[5,9]
c1=[5,-4]
d1=[4,-4]
width = c1[0] - a1[0]
height = d1[1] - a1[1]
lims = (0, 10)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

fig1 = plt.figure()
ax1 = fig1.add_subplot(111, aspect='equal')
ax1.add_patch(patches.Rectangle(a1, width, height))

a=[-6,-4]
b=[0,-4]
c=[0,-5]
d=[-6,-5]
width = c[0] - a[0]
height = d[1] - a[1]
lims = (0, 10)

#fig1 = plt.figure()
#ax1 = fig1.add_subplot(111, aspect='equal')
ax1.add_patch(patches.Rectangle(a, width, height))
 

for node in final_tree:
    (x,y,p) = node
    parent = final_tree[int(p)]
    (xp,yp,pp) = parent
    plt.plot((x,xp),(y,yp),c= 'r',linewidth=0.2) 
plt.ylim((-10, 10)) 
plt.xlim((-10,10))
plt.show()